In [27]:
import ee

try:
    ee.Initialize(project='gee-satellite-data-456115')
    print("Earth Engine     initialization successful!")
except Exception as e:
    print("Initialization failed, need authentication:", str(e))
    # authentication process
    ee.Authenticate()
    # authentication successful, then initialize again
    ee.Initialize(project='gee-satellite-data-456115')
    print("Earth Engine initialization successful!")

Earth Engine     initialization successful!


In [28]:
import ee
import geemap
import os
import datetime
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import geedim as gd

In [29]:
# use local shapefile to define the study area
def get_roi_from_shapefile(shapefile_path):
    roi_asset = geemap.shp_to_ee(shapefile_path)
    if isinstance(roi_asset, ee.FeatureCollection):
        roi_geometry = roi_asset.geometry()
    else:
        roi_geometry = roi_asset
    return roi_geometry

# load the study area boundary
shapefile_path = 'shapefile/GeldersePoort_cliped.shp'
roi = get_roi_from_shapefile(shapefile_path)

In [30]:
# Landsat数据处理函数
def calculate_landsat_indices(year):
    """计算Landsat影像的NDVI和EVI指数"""
    
    start_date = f"{year}-05-01"  # 生长季开始
    end_date = f"{year}-09-30"    # 生长季结束
    
    # 选择适当的Landsat数据集
    if year < 1999:
        # Landsat 5
        collection = 'LANDSAT/LT05/C02/T1_L2'
        nir_band = 'SR_B4'
        red_band = 'SR_B3'
        blue_band = 'SR_B1'
        swir1_band = 'SR_B5'
        swir2_band = 'SR_B7'
        green_band = 'SR_B2'
        scale_factor = 0.0000275
        offset = -0.2
    elif year < 2013:
        # Landsat 7
        collection = 'LANDSAT/LE07/C02/T1_L2'
        nir_band = 'SR_B4'
        red_band = 'SR_B3'
        blue_band = 'SR_B1'
        swir1_band = 'SR_B5'
        swir2_band = 'SR_B7'
        green_band = 'SR_B2'
        scale_factor = 0.0000275
        offset = -0.2
    else:
        # Landsat 8
        collection = 'LANDSAT/LC08/C02/T1_L2'
        nir_band = 'SR_B5'
        red_band = 'SR_B4'
        blue_band = 'SR_B2'
        swir1_band = 'SR_B6'
        swir2_band = 'SR_B7'
        green_band = 'SR_B3'
        scale_factor = 0.0000275
        offset = -0.2
    
    # 定义云掩膜函数
    def mask_clouds_landsat(image):
        qa = image.select('QA_PIXEL')
        cloud_mask = qa.bitwiseAnd(1 << 3).eq(0)  # 使用QA_PIXEL的云掩膜位
        return image.updateMask(cloud_mask)
    
    # 获取Landsat数据
    landsat = ee.ImageCollection(collection) \
        .filterBounds(roi) \
        .filterDate(start_date, end_date) \
        .filter(ee.Filter.lt('CLOUD_COVER', 20)) \
        .map(mask_clouds_landsat)
    
    # 如果数据不足，放宽云量要求
    if landsat.size().getInfo() < 5:
        landsat = ee.ImageCollection(collection) \
            .filterBounds(roi) \
            .filterDate(start_date, end_date) \
            .filter(ee.Filter.lt('CLOUD_COVER', 50)) \
            .map(mask_clouds_landsat)
        print(f"⚠️ {year}年Landsat数据较少，放宽云量限制至50%")
    
    # 应用比例因子和偏移量转换为反射率
    def apply_scale_factors(image):
        optical_bands = image.select([red_band, nir_band, blue_band, green_band, swir1_band, swir2_band]) \
            .multiply(scale_factor).add(offset)
        return image.addBands(optical_bands, None, True)
    
    landsat = landsat.map(apply_scale_factors)
    
    # 选择近红外波段质量最好的影像
    best_image = landsat.qualityMosaic(nir_band).clip(roi)
    
    # 计算NDVI: (NIR - RED) / (NIR + RED)
    ndvi = best_image.normalizedDifference([nir_band, red_band]).rename(f'NDVI_{year}')
    
    # 计算EVI: 2.5 * ((NIR - RED) / (NIR + 6*RED - 7.5*BLUE + 1))
    evi = best_image.expression(
        '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))',
        {
            'NIR': best_image.select(nir_band),
            'RED': best_image.select(red_band),
            'BLUE': best_image.select(blue_band)
        }
    ).rename(f'EVI_{year}')
    
    # 重命名波段以保持一致性
    renamed_bands = best_image.select(
        [blue_band, green_band, red_band, nir_band, swir1_band, swir2_band],
        [f'Blue_{year}', f'Green_{year}', f'Red_{year}', f'NIR_{year}', f'SWIR1_{year}', f'SWIR2_{year}']
    )
    
    return {
        'Original': renamed_bands,
        'NDVI': ndvi,
        'EVI': evi
    }

In [31]:
# Sentinel-2数据处理函数
def calculate_sentinel_indices(year):
    """计算Sentinel-2影像的NDVI和EVI指数"""
    
    start_date = f"{year}-05-01"  # 标准窗口
    end_date = f"{year}-09-30"
    
    # 定义云掩膜函数 - Sentinel-2
    def mask_clouds_s2(image):
        # 使用SCL波段进行云检测
        # SCL值: 1=饱和/缺失, 2=暗阴影, 3=云阴影, 4=植被, 5=裸土, 6=水, 7=未分类, 
        # 8=云中等概率, 9=云高概率, 10=薄卷云, 11=雪/冰
        scl = image.select('SCL')
        cloud_mask = scl.neq(9).And(scl.neq(8)).And(scl.neq(10)).And(scl.neq(1))
        
        # 返回掩膜后的图像，只保留我们需要的波段
        return image.select(['B2', 'B3', 'B4', 'B8', 'B11', 'B12']).updateMask(cloud_mask)
    
    # 获取Sentinel-2数据
    sentinel = ee.ImageCollection('COPERNICUS/S2_SR') \
        .filterBounds(roi) \
        .filterDate(start_date, end_date) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
        .map(mask_clouds_s2)
    
    # 检查数据量，如果太少则放宽云量限制
    if sentinel.size().getInfo() < 5:
        sentinel = ee.ImageCollection('COPERNICUS/S2_SR') \
            .filterBounds(roi) \
            .filterDate(start_date, end_date) \
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 50)) \
            .map(mask_clouds_s2)
        print(f"⚠️ {year}年Sentinel-2数据较少，放宽云量限制至50%")
    
    # 使用近红外波段(B8)质量最好的像素
    best_image = sentinel.qualityMosaic('B8').clip(roi)
    
    
    # 计算NDVI: (NIR - RED) / (NIR + RED)
    ndvi = best_image.normalizedDifference(['B8', 'B4']).rename(f'NDVI_{year}')
    
    # 计算EVI: 2.5 * ((NIR - RED) / (NIR + 6*RED - 7.5*BLUE + 1))
    evi = best_image.expression(
        '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))',
        {
            'NIR': best_image.select('B8'),
            'RED': best_image.select('B4'),
            'BLUE': best_image.select('B2')
        }
    ).rename(f'EVI_{year}')
    
    # 重命名波段以保持一致性
    renamed_bands = best_image.select(
        ['B2', 'B3', 'B4', 'B8', 'B11', 'B12'],
        [f'Blue_{year}', f'Green_{year}', f'Red_{year}', f'NIR_{year}', f'SWIR1_{year}', f'SWIR2_{year}']
    )
    
    return {
        'Original': renamed_bands,
        'NDVI': ndvi,
        'EVI': evi
    }

In [32]:
def download_satellite_data(start_year=1993, end_year=2024, parent_folder='GEEpreprocessing'):
    """
    下载特定年份的卫星数据，计算NDVI和EVI指数
    所有数据统一采用30m分辨率和RD New坐标系
    
    Args:
        start_year: 起始年份
        end_year: 结束年份
        parent_folder: 保存结果的文件夹
    """
    
    print(f"开始处理{start_year}-{end_year}年的卫星数据...")
    
    # 确保输出文件夹存在
    if not os.path.exists(parent_folder):
        os.makedirs(parent_folder)
    
    # 处理每一年的数据
    for year in range(start_year, end_year + 1):
        print(f"处理{year}年数据...")
        
        # 根据年份选择不同的卫星数据
        if year < 2017:
            indices = calculate_landsat_indices(year)
        else:
            indices = calculate_sentinel_indices(year)
        
        # 所有数据统一使用30m分辨率
        resolution = 30
        
        # 导出原始波段
        # filename = os.path.join(parent_folder, f"Original_{year}.tif")
        # geemap.download_ee_image(
        #     indices['Original'], 
        #     filename=filename,
        #     region=roi,
        #     scale=resolution,
        #     crs='EPSG:28992'  # 使用RD New坐标系
        # )
        # print(f"✅ {year}年原始波段导出完成")
        
        # 导出NDVI
        filename = os.path.join(parent_folder, f"NDVI_{year}.tif")
        geemap.download_ee_image(
            indices['NDVI'], 
            filename=filename,
            region=roi,
            scale=resolution,
            crs='EPSG:28992'
        )
        print(f"✅ {year}年NDVI导出完成")
        
        # 导出EVI
        filename = os.path.join(parent_folder, f"EVI_{year}.tif")
        geemap.download_ee_image(
            indices['EVI'], 
            filename=filename,
            region=roi,
            scale=resolution,
            crs='EPSG:28992'
        )
        print(f"✅ {year}年EVI导出完成")
    
    print(f"\n所有数据处理完成，文件已保存至{parent_folder}目录")

In [26]:
download_satellite_data(1993, 2024, 'GEEpreprocessing')

开始处理2015-2015年的卫星数据...
处理2015年数据...
⚠️ 2015年Sentinel-2数据较少，放宽云量限制至50%


EEException: Image.normalizedDifference: No band named 'B8'. Available band names: [].